In [1]:
import tensorflow as tf
import os

In [2]:
base_dir='./content/fruits-360/Training'

In [3]:
_IMAGE_SIZE=224
_BATCH_SIZE=64


In [4]:
# splitted to training and validation
data_gen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1
)

In [5]:
# 90% train =>

train_gen=data_gen.flow_from_directory(
    base_dir,
target_size=(_IMAGE_SIZE,_IMAGE_SIZE),
batch_size=_BATCH_SIZE,
subset='training'
)

Found 60955 images belonging to 131 classes.


In [6]:
# check for 131 keys
#
# train_gen.class_indices 

In [7]:
lables='\n'.join(sorted(train_gen.class_indices.keys()))
with open('labels.txt','w') as f:
    f.write(lables)

#del from memory
del lables

In [8]:
# 10% validation =>

val_gen=data_gen.flow_from_directory(
    base_dir,
target_size=(_IMAGE_SIZE,_IMAGE_SIZE),
batch_size=_BATCH_SIZE,
subset='validation'
)

Found 6737 images belonging to 131 classes.


In [9]:
_IMG_SHAPE=(_IMAGE_SIZE,_IMAGE_SIZE,3)
base_model=tf.keras.applications.MobileNetV2(
    input_shape=_IMG_SHAPE,
    include_top=False,
    weights='imagenet'
    )

In [10]:
base_model.trainable=False

In [11]:
#short import to make the next command shorter
from tensorflow.keras import layers

# 5 = num of classes
model=tf.keras.Sequential([
    base_model,
layers.Conv2D(32,3,activation='relu'),
layers.Dropout(0.2),
layers.GlobalAveragePooling2D(),
layers.Dense(131,
    activation='softmax'
)
])

In [12]:
model.compile(
optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',
metrics=['accuracy']
 )

In [ ]:
#fitting the model
epochs=10
history=model.fit(
    train_gen,
    epochs=epochs,
    steps_per_epoch=200,
    validation_data=val_gen
    )

In [ ]:
saved_model_dir=''
tf.saved_model.save(model,saved_model_dir)
converter=tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model=converter.convert()

with open('model.tflite','wb') as f:
    f.write(tflite_model)

In [ ]:
# in the project dir there will be many new files 
# we want only the labels.txt and model.tflite files.

In [ ]:
# in the video he used commands to download from the colab website
# i've done it localy, so just copy to the assets folder in the flutter projects.